## Function decorators and closures

In [2]:
@decorate
def target():
    print('running target()')
    
# Has the same effect as writing this:

def target():
    print('running target()')

target = decorate(target)

NameError: name 'decorate' is not defined

In [3]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')
    
target()

running inner()


In [4]:
target

<function __main__.deco.<locals>.inner()>

In [5]:
# registration.py
# When Python executes decorators
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()
    
if __name__ == '__main__':
    main()

running register(<function f1 at 0x1042050d0>)
running register(<function f2 at 0x104205488>)
running main()
registry -> [<function f1 at 0x1042050d0>, <function f2 at 0x104205488>]
running f1()
running f2()
running f3()


### Closures 闭包

In [9]:
# average_oo.py
# a class to calculate a running average
class Averager():
    
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

avg = Averager()
avg(10)

10.0

In [10]:
avg(11)

10.5

In [11]:
avg(12)

11.0

In [13]:
# average.py 
# a higher-order function to calculate a running average
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    
    return averager

avg = make_averager()
avg(10)

10.0

In [14]:
avg(11)

10.5

In [15]:
avg.__code__.co_varnames

('new_value', 'total')

In [16]:
avg.__code__.co_freevars

('series',)

In [17]:
avg.__closure__

(<cell at 0x10417c258: list object at 0x1041e8b88>,)

In [21]:
avg.__closure__[0].cell_contents

[10, 11]

In [22]:
# The nonlocal declaration
def make_averager():
    count = 0
    total = 0
    
    def averager(new_Value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager

### Implementing a simple decorator

In [26]:
# clockdeco_demo.py
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) - %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.223)
    print('*' * 40, 'Calling facorial(6)')
    print('6! =', factorial(10))

**************************************** Calling snooze(.123)
[0.22509703s] snooze(0.223) - None
**************************************** Calling facorial(6)
[0.00000043s] factorial(1) - 1
[0.00003336s] factorial(2) - 2
[0.00037343s] factorial(3) - 6
[0.00041433s] factorial(4) - 24
[0.00044135s] factorial(5) - 120
[0.00045826s] factorial(6) - 720
[0.00047473s] factorial(7) - 5040
[0.00050292s] factorial(8) - 40320
[0.00052780s] factorial(9) - 362880
[0.00054577s] factorial(10) - 3628800
6! = 3628800


In [28]:
# clockdeco.py
# An improved clock decorator, support keyword argument, it masks the __name__ 
# and __doc__ of the decorated function
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args: 
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.223)
    print('*' * 40, 'Calling facorial(6)')
    print('6! =', factorial(10))

**************************************** Calling snooze(.123)
[0.22513604s] snooze(0.223) -> None
**************************************** Calling facorial(6)
[0.00000000s] factorial(1) -> 1
[0.00003099s] factorial(2) -> 2
[0.00020409s] factorial(3) -> 6
[0.00022125s] factorial(4) -> 24
[0.00024509s] factorial(5) -> 120
[0.00028014s] factorial(6) -> 720
[0.00030684s] factorial(7) -> 5040
[0.00033283s] factorial(8) -> 40320
[0.00036788s] factorial(9) -> 362880
[0.00039506s] factorial(10) -> 3628800
6! = 3628800


### functools.lru_cache  
LRU stand for Least Recently Used  
> It implements memoization

In [29]:
from clockdeco import clock

@clock 
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

print(fibonacci(6))

[0.00000000s] fibonacci(0) -> 0
[0.00000119s] fibonacci(1) -> 1
[0.00049305s] fibonacci(2) -> 1
[0.00000095s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00007510s] fibonacci(2) -> 1
[0.00015306s] fibonacci(3) -> 2
[0.00070024s] fibonacci(4) -> 3
[0.00000095s] fibonacci(1) -> 1
[0.00000095s] fibonacci(0) -> 0
[0.00000095s] fibonacci(1) -> 1
[0.00004005s] fibonacci(2) -> 1
[0.00008297s] fibonacci(3) -> 2
[0.00000095s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00003886s] fibonacci(2) -> 1
[0.00000119s] fibonacci(1) -> 1
[0.00000095s] fibonacci(0) -> 0
[0.00000095s] fibonacci(1) -> 1
[0.00007892s] fibonacci(2) -> 1
[0.00046802s] fibonacci(3) -> 2
[0.00087905s] fibonacci(4) -> 3
[0.00101209s] fibonacci(5) -> 5
[0.00176096s] fibonacci(6) -> 8
8


In [30]:
import functools
from clockdeco import clock

@functools.lru_cache()
@clock 
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

print(fibonacci(6))

[0.00000095s] fibonacci(0) -> 0
[0.00000119s] fibonacci(1) -> 1
[0.00009680s] fibonacci(2) -> 1
[0.00000095s] fibonacci(3) -> 2
[0.00021100s] fibonacci(4) -> 3
[0.00000191s] fibonacci(5) -> 5
[0.00029492s] fibonacci(6) -> 8
8


In [31]:
import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

htmlize('Heimlich & Co.\n- a game')

'<pre>&#x27;Heimlich &amp; Co.\\n- a game&#x27;</pre>'

In [32]:
htmlize(42)

'<pre>42</pre>'

In [33]:
print(htmlize(['alpha', 66, {3, 2, 1}]))

<pre>[&#x27;alpha&#x27;, 66, {1, 2, 3}]</pre>


### Generic function with single dispatch

In [34]:
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)

@htmlize.register(numbers.Integral)
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'


In [35]:
htmlize(42)

'<pre>42 (0x2a)</pre>'

In [36]:
print(htmlize(['alpha', 66, {3, 2, 1}]))

<ul>
<li><p>alpha</p></li>
<li><pre>66 (0x42)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>


### A parametrized registration decorator

In [38]:
registry = set()

def register(active=True):
    def decorate(func):
        print('running register(active=%s)->decorate(%s)' % (active, func))
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func
    return decorate

@register(active=False)
def f1():
    print('running f1()')

@register()
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')
    


running register(active=False)->decorate(<function f1 at 0x104247840>)
running register(active=True)->decorate(<function f2 at 0x104205f28>)


In [39]:
registry

{<function __main__.f2()>}

In [42]:
#clockdeco_param.py
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args):
            t0 = time.time()
            _result = func(*_args)
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate

if __name__ == '__main__':
    @clock()
    def snooze(seconds):
        time.sleep(seconds)
        
    for i in range(3):
        snooze(.123)
        

[0.12668276s] snooze(0.123) -> None
[0.12677693s] snooze(0.123) -> None
[0.12417102s] snooze(0.123) -> None


In [43]:
from clockdeco_param import clock

@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze: 0.12333369255065918s
snooze: 0.12819314002990723s
snooze: 0.12676286697387695s
